### Mount GDrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Install and Import SAM

In [2]:
%cd "/content/drive/MyDrive/Final IDCC-SAM/"

!git clone https://github.com/facebookresearch/segment-anything.git
!cd segment-anything; pip install -e.

!pip install git+https://github.com/facebookresearch/segment-anything.git

/content/drive/MyDrive/Final IDCC-SAM
fatal: destination path 'segment-anything' already exists and is not an empty directory.
Obtaining file:///content/drive/MyDrive/Final%20IDCC-SAM/segment-anything
  Preparing metadata (setup.py) ... done
  Running setup.py develop for segment-anything
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-z8ofl9a2
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-z8ofl9a2
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36590 sha256=45d5deebe7307f07e3bacb0221b915f198149b6af026e8c07a70b08a4d467bfa
  Stored in directory: /tmp/pip-ephem-wheel-cache-dtp1q5j_/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built

In [3]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry,  SamAutomaticMaskGenerator, SamPredictor

In [4]:
if not os.path.exists("/content/drive/MyDrive/Final IDCC-SAM/segment-anything/sam_vit_h_4b8939.pth"):
  !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -O /content/drive/MyDrive/Final IDCC-SAM/segment-anything/sam_vit_h_4b8939.pth

### Define SAM Mask Generator Functions

In [5]:
# Define function to visualize annotations (masks) and provide functionality
# to use the Segment Anything Model (SAM) for generating automatic masks.

import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# Path to the SAM model checkpoint, downloaded during installation
sam_checkpoint = "/content/drive/MyDrive/Final IDCC-SAM/segment-anything/sam_vit_h_4b8939.pth"

#Use the visual transformer backbone
model_type = "vit_h"

device = "cuda"

# Load the SAM model from the registry using the provided checkpoint
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

# Create a mask generator using the SAM model
mask_generator = SamAutomaticMaskGenerator(
     model=sam,
     pred_iou_thresh=0.8
 )
def show_anns(anns):

  #Display annotations (masks) on the current plot.
    #Parameters:
        #anns (list): List of annotations, each containing a segmentation mask.

    if len(anns) == 0:
        return

    # Sort annotations based on area in descending order
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    polygons = []
    color = []
    # Plot each annotation
    for ann in sorted_anns:
        m = ann['segmentation'] # Segmentation mask
        img = np.ones((m.shape[0], m.shape[1], 3)) # Create an RGB image
        color_mask = np.random.random((1, 3)).tolist()[0] # Generate a random color
        for i in range(3):
            img[:,:,i] = color_mask[i] # Assign the color to each channel
        ax.imshow(np.dstack((img, m*0.35))) # Overlay the mask on the image with transparency

### IDCC-SAM: Zero-Shot Cell Counting -- Run on all images in a loop and generate masks

In [6]:
import cv2
import numpy as np
import os
import pandas as pd

# Path to the folder containing the images
folder_path = '/content/drive/MyDrive/Final IDCC-SAM/Dataset/Rebuttal/ADC/Test/images/'

# Get the list of image filenames and sort them in ascending order
image_filenames = sorted(os.listdir(folder_path))

# Load the ground truth Excel file
excel_file_path = '/content/drive/MyDrive/Final IDCC-SAM/Dataset/Rebuttal/ADC/ground_truth.csv'
ground_truth_df = pd.read_csv(excel_file_path)

# Create lists to store our prediction results summary information at the end
summary_data = []

# Loop through each image
for filename in image_filenames:
    # Load the original image
    image_path = os.path.join(folder_path, filename)
    original_image = cv2.imread(image_path, 0)

    # Convert image to RGB and then Generate masks
    image= cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    masks = mask_generator.generate(image)

    # For our evaluation, let's extract number of cells from ground truth Excel files
    image_name = os.path.splitext(filename)[0]  # Remove file extension
    matching_row = ground_truth_df[ground_truth_df['image name'] == image_name + '.png']
    num_cells_gt = matching_row['ground truth'].values[0] if not matching_row.empty else 'Unknown'

    # Calculate the absolute difference between ground truth and predicted cell count
    abs_diff = abs(num_cells_gt - len(masks))
    abs_diff_str = f'Abs Diff: {abs_diff}' if num_cells_gt != 'Unknown' else 'Abs Diff: Unknown'

    # Calculate the absolute percentage accuracy
    abs_percentage_accuracy = f'{(1 - abs_diff / num_cells_gt) * 100:.2f}%' if num_cells_gt != 'Unknown' and num_cells_gt != 0 else 'Unknown'

    # Add summary data to list
    summary_data.append({
        'Image Name': image_name,
        'Ground Truth': num_cells_gt,
        'Prediction': len(masks),
        'Raw Absolute Error': abs_diff
    })

# Create DataFrame from summary data
summary_df = pd.DataFrame(summary_data)

# Save summary data to Excel file
summary_excel_path = '/content/drive/MyDrive/Final IDCC-SAM/Rebuttal/ADC/IDCC-SAM/IDCC-SAM_prediction_summary_ADC.xlsx'
summary_df.to_excel(summary_excel_path, index=False)

Compute MAE and AAE

In [7]:
import pandas as pd

# Path to the Excel file containing the results
results_excel_file = '/content/drive/MyDrive/Final IDCC-SAM/Rebuttal/ADC/IDCC-SAM/IDCC-SAM_prediction_summary_ADC.xlsx'

# Load the results from the Excel file
results_df = pd.read_excel(results_excel_file)

# Create the new column "AAE <=10"
results_df['AAE <=10'] = results_df['Raw Absolute Error'].apply(lambda x: 'Yes' if x <= 10 else 'No')

sorted_results_df = results_df.sort_values(by='Raw Absolute Error', ascending=True)

# Save the updated DataFrame back to the Excel file
results_df.to_excel(results_excel_file, index=False)

# Display the updated DataFrame
print(sorted_results_df)


                                     Image Name  Ground Truth  Prediction  \
5   28900-25500_GTEX-11ZUS_Adipose-Subcutaneous           136         137   
32  34000-11900_GTEX-11EQ8_Adipose-Subcutaneous           129         131   
20   30600-6800_GTEX-15DYW_Adipose-Subcutaneous           176         179   
21  32300-10200_GTEX-1F6RS_Adipose-Subcutaneous           121         117   
15  30600-11900_GTEX-14B4R_Adipose-Subcutaneous           147         152   
46   35700-13600_GTEX-YEC3_Adipose-Subcutaneous            95         101   
33  34000-13600_GTEX-131XF_Adipose-Subcutaneous           149         143   
49   35700-6800_GTEX-1F7RK_Adipose-Subcutaneous           180         173   
44  35700-11900_GTEX-13S7M_Adipose-Subcutaneous           143         136   
42   34000-8500_GTEX-1CB4I_Adipose-Subcutaneous           129         122   
47    35700-5100_GTEX-QXCU_Adipose-Subcutaneous           142         134   
43    34000-8500_GTEX-WK11_Adipose-Subcutaneous           100          90   

### For Visualization purpose, generate and save the final segmentation output images from SAM

Segment with multicolor

In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

# Function to generate random colors
def generate_random_colors(num_colors):
    return np.random.rand(num_colors, 3)  # Generate random RGB values in the range [0, 1]


# Path to the folder containing the images
folder_path = '/content/drive/MyDrive/Final IDCC-SAM/Dataset/Rebuttal/ADC/Test/images/'

# Get the list of image filenames and sort them in ascending order
image_filenames = sorted(os.listdir(folder_path))

# Load the ground truth Excel file
excel_file_path = '/content/drive/MyDrive/Final IDCC-SAM/Dataset/Rebuttal/ADC/ground_truth.csv'  # Replace with the actual path
ground_truth_df = pd.read_csv(excel_file_path)

# Create lists to store prediction summary information
summary_data = []

# Calculate the number of rows and columns for subplots
num_images = len(image_filenames)
num_columns = 2
num_rows = num_images

# Create a directory to save the plots as images
output_folder = '/content/drive/MyDrive/Final IDCC-SAM/Rebuttal/ADC/IDCC-SAM/Test results_IDCC-SAM'
os.makedirs(output_folder, exist_ok=True)

#for i, filename in enumerate(image_filenames[1:10], 1):
for i, filename in enumerate(image_filenames, 1):
    # Load the original image
    image_path = os.path.join(folder_path, filename)
    original_image = cv2.imread(image_path, 0)

    # Load the masks
    image= cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    masks = mask_generator.generate(image)

    # Extract number of cells from ground truth Excel file
    image_name = os.path.splitext(filename)[0]  # Remove file extension
    matching_row = ground_truth_df[ground_truth_df['image name'] == image_name + '.png']
    num_cells_gt = matching_row['ground truth'].values[0] if not matching_row.empty else 'Unknown'

    # Calculate the absolute difference between ground truth and predicted cells
    abs_diff = abs(num_cells_gt - len(masks))
    num_cells = f'Pred: {len(masks)}' if num_cells_gt != 'Unknown' else 'Abs Diff: Unknown'
    abs_diff_str = f'Abs Diff: {abs_diff}' if num_cells_gt != 'Unknown' else 'Abs Diff: Unknown'

    # Add summary data to list
    summary_data.append({
        'Image Name': image_name,
        'Ground Truth': num_cells_gt,
        'Prediction': len(masks),
        'Absolute Difference': abs_diff
    })

    # Create the plot
    plt.figure(figsize=(8, 4))  # Adjusted figsize
    plt.subplot(1, 2, 1)
    plt.imshow(original_image, cmap='gray')
    plt.title(f'Original Image\nNumber of Cells: {num_cells_gt}')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(original_image, cmap='gray')
    plt.title('Mask \n' + num_cells + '\n' + abs_diff_str)
    colors = generate_random_colors(len(masks))  # Generate random colors
    for j, v in enumerate(masks):
        x = v['bbox'][0] + v['bbox'][2] / 2
        y = v['bbox'][1] + v['bbox'][3] / 2
        color = colors[j]  # Assign a unique color to each mask
        plt.plot(x, y, marker="o", markersize=4, markeredgecolor="white", markerfacecolor=color)
    plt.axis('off')

    # Save the plot as an image
    output_filename = os.path.join(output_folder, f'{image_name}_plot.tiff')
    plt.savefig(output_filename, bbox_inches='tight')
    plt.close()


### End